In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from peft import PeftModel, PeftConfig

# Load LoRA config
peft_config = PeftConfig.from_pretrained("t5_finetuned_best")

# Load base model
base_model = T5ForConditionalGeneration.from_pretrained(peft_config.base_model_name_or_path)

# Load LoRA adapters
model = PeftModel.from_pretrained(base_model, "t5_finetuned_best")

tokenizer = T5Tokenizer.from_pretrained(peft_config.base_model_name_or_path)


2025-04-15 00:12:20.643392: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 00:12:20.658398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744675940.674230 2718034 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744675940.679324 2718034 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744675940.691492 2718034 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
for module in model.named_modules():
    print(module[0], module[1])

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Example input
# input_text = "Please answer the following question: What are two core biomarkers of Alzheimer's Disease?"
# input_text = "extract gene and species: <extra_id_0> is expressed in Mus musculus"
# input_text = "<extra_id_0> is a common Alzheimer's indicative gene."
input_text = "The mitochondria is the <extra_id_0> of the cell."
# input_text = "The gene <extra_id_0> encodes a component of the γ-secretase complex and is frequently mutated in early-onset Alzheimer's disease."

# Tokenize
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate output
with torch.no_grad():
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=256,   # Or whatever you want
        num_beams=4,      # For higher quality output
    )

output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated Output:", output)

Generated Output: energy powerhouse
